<img style='float:left' src='https://i.ibb.co/pzSzM5y/logo-200x200.png' width='200px'>
<br><br><br><br>

**Work with data & machine learning models**

* easily store data in a high-performance data cluster (MongoDB)
* store your fitted or unfitted scikit-learn models
* run predictions on the compute cluster directly from stored data
* store & use remote data (ftp, http, s3)

**Easily use compute resources in the cluster**

* fit models in the compute cluster, in parallel
* perform grid search
* all asynchronously

**Share data, notebooks**

* write, store & share notebooks directly online, no setup required
* run jobs on a regular schedule
* share notebooks and data across users

**Automatic REST API for any client**

* datasets
* models
* jobs (reports)
* arbitrary custom scripts (python)

**On-Premise or On-Cloud Custom Installation**

* customizable backends (e.g. Spark, R, SAS)
* custom runtimes (e.g. dask, Spark)
* arbitrary data storage extensions API
* custom data types extensions API
* native-Python data streaming API (like Spark Streaming, much simpler)

In [ ]:
!pip install --user -U pandas_datareader tqdm

In [ ]:
import omegaml as om 
om.setup()

In [ ]:
# list datasets stored in cluster
om.datasets.list()

In [ ]:
# list models stored in clusters
om.models.list()

In [ ]:
# list jobs & results stored in cluster
om.jobs.list()

In [ ]:
# Enterprise Edition
# list custom scripts stored in cluster
# om.scripts.list()

In [ ]:
# store any python data
om.datasets.put(['any data'], 'mydata')
om.datasets.get('mydata')

In [ ]:
# store numpy arrays and pandas dataframes
import pandas as pd
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
data = pd.DataFrame(X)
data['y'] = y
data.head()
om.datasets.put(data, 'iris')
om.datasets.get('iris').head()

In [ ]:
# Enterprise Edition
# store remote datasets as a reference (no copy)
# om.datasets.put('http://data.cityofnewyork.us/api/views/kku6-nxdu/rows.csv?accessType=DOWNLOAD', 'demographics')
# om.datasets.get('demographics')

In [ ]:
# store financial time series including indicies
%matplotlib inline
import pandas as pd
import pandas_datareader.data as web
import datetime

start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2018, 1, 31)
prices = web.DataReader("GOOGL", 'yahoo', start, end)
prices.head()

In [ ]:
# get data back in their original format
om.datasets.put(prices, 'google', append=False)
prices = om.datasets.get('google')
prices.head()

In [ ]:
# filter data in the database -- notice the nice syntax
%time om.datasets.get('google', Close__gte=900, Close__lte=920)

In [ ]:
# filter & aggregate data locally (let's make it large)
from tqdm import tqdm
N = 1e6
ldf_google_large = om.datasets.getl('google-large')
dupl = int((N - len(ldf_google_large or [])) / len(prices) + 1)
for i in tqdm(range(dupl)):
    om.datasets.put(prices, 'google-large')
print("google-large has {} records".format(len(om.datasets.getl('google-large'))))

In [ ]:
# filter & aggregate data locally (let's make it large)
def getdata():
  data = om.datasets.get('google-large')
  return data[(data.Close >= 900) & (data.Close <= 920)].mean() 

%time getdata()

In [ ]:
# filter and aggregate by database - 2-3x faster
%time om.datasets.getl('google-large', Close__gte=900, Close__lte=920).mean().iloc[0]

In [ ]:
# index based access by loading data first
def getdata():
    dfx = om.datasets.get('google-large')
    return dfx.loc[pd.to_datetime('2017-01-03')]
%time getdata()

In [ ]:
# index-based access directly in database
dfx = om.datasets.getl('google-large')
%time dfx.loc[pd.to_datetime('2017-01-03')].value

In [ ]:
# train models locally
%matplotlib inline
import pandas as pd 

from sklearn.svm import SVR

prices = om.datasets.get('google')
X = prices[['High', 'Low']].rolling(5).mean().dropna()
y = prices.iloc[4:]['Close']
print(X.shape, y.shape)

train_loc = X.shape[0] // 2

model = SVR(kernel='linear', tol=0.1)
model.fit(X.iloc[0:train_loc], y.iloc[0:train_loc])

r2 = model.score(X, y)
yhat = pd.DataFrame({'yhat': model.predict(X[train_loc:])})
yhat.index = X.index[train_loc:]

ax = prices.iloc[train_loc:]['Close'].plot()
yhat.plot(color='r', ax=ax)

In [ ]:
# predict remotely

# store models and new data
om.models.put(model, 'google-predict')
om.datasets.put(X[train_loc:], 'google-rolling', append=False)

# then predict remotely
pred = om.runtime.model('google-predict').predict('google-rolling[High,Low]').get()

# show results
pred = pd.DataFrame({'yhat': pred}, index=range(len(pred)))
actual = om.datasets.get('google[Close]').iloc[train_loc:]
pred.index = actual.index[:len(pred)]
ax = actual.plot()
pred.plot(color='r', ax=ax)

In [ ]:
# we can also train remote
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

df = pd.DataFrame(X)
df['y'] = y

from sklearn.cluster import KMeans
model = KMeans(n_clusters=8)

# fit & predict remote
om.models.drop('iris-model', True)
om.models.put(model, 'iris-model')
om.runtime.model('iris-model').fit(X, y).get()

# get back remote fitted model and show results
model = om.models.get('iris-model')
labels = model.labels_

fig = plt.figure(figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(X[:, 3], X[:, 0], X[:, 2],
               c=labels.astype(np.float), edgecolor='k')
fig.show()

In [ ]:
# we store lots of information on models
om.models.metadata('iris-model').attributes

In [ ]:
# perform gridsearch on cluster
om.datasets.put(df, 'iris', append=False)
params = {
            'n_clusters': range(1,8),
         }
om.runtime.model('iris-model').gridsearch('iris[^y]', 'iris[y]', parameters=params).get()

In [ ]:
# see what gridsearch results we have
gsresult = om.models.metadata('iris-model')['attributes']['gridsearch']

In [ ]:
# look at gridsearch results
gsModel = gsresult[0]['gsModel']
gs = om.models.get(gsModel)
gs.best_estimator_

In [ ]:
# use the model REST API 
import requests
from omegaml.client.auth import OmegaRestApiAuth
import omegaml as om 

# -- setup authentication and API URL
auth = OmegaRestApiAuth.make_from(om)
url = getattr(om.defaults, 'OMEGA_RESTAPI_URL', 'http://localhost:5000')
modelname = 'iris-model'
dataset = 'iris'
# -- prepare dataset
om.datasets.put(pd.DataFrame(X), 'iris', append=False)
# -- call REST API
print('Requesting from', url)
resp = requests.put('{url}/api/v1/model/{modelname}/predict?datax={dataset}'.format(**locals()), auth=auth)
resp.json()

In [ ]:
# use the datasets REST API 
import requests
import time
time.sleep(5)

print('Requesting from', url)
resp = requests.get('{url}/api/v1/dataset/{dataset}'.format(**locals()), auth=auth)
resp.json()

In [ ]:
# Enterprise Edition
# deploy lambda-style arbitrary algorithms
# om.scripts.put('pkg:///app/omegapkg/demo/helloworld/', 'helloworld')

In [ ]:
# Enterprise Edition
# run lambdas
# from datetime import datetime
# dtnow = datetime.now().isoformat()
# om.runtime.script('helloworld').run(foo=dtnow).get()

In [ ]:
# Enterprise Edition
# use REST API to run lambdas
# import requests
# from omegacommon.auth import OmegaRestApiAuth
# auth = OmegaRestApiAuth(**auth_config)
# resp = requests.post('https://omegaml.omegaml.io/api/v1/script/helloworld/run/', 
#                      params=dict(foo=dtnow), auth=auth)
# resp.json()

In [ ]:
# run jobs (python notebooks) online
if 'scheduled-report.ipynb' in om.jobs.list():
    om.runtime.job('scheduled-report').run()
    om.jobs.list()

## Enterprise Edition

### per-user online dashboard 
http://omegaml.omegaml.io/dashboard
    
### per-user online notebook automated setup
http://omjobs.omegaml.io/